<a href="https://colab.research.google.com/github/txvuong150764/Sentence_Semantic_phoBERT/blob/master/QS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 76.1 MB/s 
     |████████████████████████████████| 182 kB 80.7 MB/s 


In [2]:
from torch.utils.data import Dataset, DataLoader

In [3]:
from tensorflow import keras
from keras.layers import Dense
import tensorflow as tf
import torch.nn as nn
import numpy as np
from tqdm import tqdm

In [84]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install vncorenlp
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP('VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx500m')

def word_segment(text):
    word_segmented_text = annotator.tokenize(text) 
    sentences = []
    for sentence in word_segmented_text:
        sentences.append(' '.join(sentence))
    return ' '.join(sentences)

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
s = word_segment("Ngày quốc khánh của việt nam là 2/9")
s

'Ngày quốc_khánh của việt_nam là 2/9'

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 6
max_length = 128

In [8]:
import pandas as pd

dataset_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/dev.csv')

In [9]:
for i in range(len(dataset_df)):
    dataset_df['question 1'][i] = word_segment(dataset_df['question 1'][i])
    dataset_df['question 2'][i] = word_segment(dataset_df['question 2'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(dataset_df, test_size=0.1, random_state=42)

In [11]:
valid_df

,question 1,question 2,similarity
53261,lý thái tổ mất vào ngày_tháng năm nào,isaac newton mất vào ngày nào,False
123871,vì sao phải mặc quần_lót,vì sao lại mà phải mặc quần_lót,True
28417,ngày truyền_thống của quân_lực việt_nam cộng_h...,ngày truyền_thống của quân_lực việt_nam cộng_h...,False
33182,phim titanic phát_hành năm nào,bộ manga nào có lượt phát_hành kỷ_lục năm 2008,False
83411,bảy kỳ_quan thế_giới mới được lựa_chọn như_thế...,điều_kiện lựa_chọn thẩm_phán của mỗi_một viện,False
...,...,...,...
100534,trên lá cờ eu có bao_nhiêu ngôi_sao,trên lá cờ eu có bao_nhiêu ngôi_sao,True
67588,gang có những tính_chất gì,gang được sản_xuất như_thế_nào,False
74211,trạng_nguyên nguyễn_trực quê ở đâu,nguyễn_trung_trực quê gốc ở đâu,False
57642,quốc_huy bồ_đào_nha được thông_qua vào ngày nào,bồ_đào_nha giáp với những nước nào,False


In [12]:
for i in range(len(test_df)):
  test_df['question 1'][i] = word_segment(test_df['question 1'][i])
  test_df['question 2'][i] = word_segment(test_df['question 2'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
print(test_df['question 1'][0], test_df['question 2'][0])

văn_hoá việt_nam có từ bao_giờ tên gọi hai chữ việt_nam và quốc_hiệu việt_nam có từ bao_giờ


In [14]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
class EquavilenceDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        """
        To customize dataset, inherit from Dataset class and implement
        __len__ & __getitem__
        __getitem__ should return 
            data:
                input_ids
                attention_masks
                text
                targets
        """
        row = self.df.iloc[index]
        text1, text2, label = self.get_input_data(row)

        # Encode_plus will:
        # (1) split text into token
        # (2) Add the '[CLS]' and '[SEP]' token to the start and end
        # (3) Truncate/Pad sentence to max length
        # (4) Map token to their IDS
        # (5) Create attention mask
        # (6) Return a dictionary of outputs
        encoding = self.tokenizer(
            text1,
            text2,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
        
        return {
            'text1': text1,
            'text2': text2,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_masks': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long),
        }


    def labelencoder(self,text):
        if text==True:
            return 1
        elif text==False:
            return 0
        else:
            return 2

    def get_input_data(self, row):
        # Preprocessing: {remove icon, special character, lower}
        text1 = row['question 1']
        text2 = row['question 2']
        label = self.labelencoder(row['similarity'])

        return text1, text2, label

In [16]:
tokenizer_datasets = EquavilenceDataset(train_df, tokenizer, max_len=128)
tokenizer_validations = EquavilenceDataset(valid_df, tokenizer, max_len=128)

In [17]:
tokenizer_datasets[1]

{'text1': 'gdp của nga năm 2016 là bao_nhiêu',
 'text2': 'gdp của paris 2016 là bao_nhiêu',
 'input_ids': tensor([    0, 45134,  2380,     7, 11018,    29,   484,     8,  1823,     2,
             2, 45134,  2380,     7,  5877,  2573,   484,     8,  1823,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,    

In [18]:
tokenizer_validations[3]

{'text1': 'phim titanic phát_hành năm nào',
 'text2': 'bộ manga nào có lượt phát_hành kỷ_lục năm 2008',
 'input_ids': tensor([    0,   266,  2081,  6691,  4933,  1377,    29,   142,     2,     2,
           215, 23420,   142,    10,  1030,  1377,  1888,    29,  1531,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1

In [19]:
from torch.utils.data import Dataset, DataLoader

train_loader = DataLoader(tokenizer_datasets, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(tokenizer_validations, batch_size=16, shuffle=True, num_workers=2)

In [20]:
class EquivalenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EquivalenceClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [21]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in tqdm(train_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

def eval(test_data = False):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        data_loader = test_loader if test_data else valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())
    
    if test_data:
        print(f'Test Accuracy: {correct.double()/len(test_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(test_loader.dataset)
    else:
        print(f'Valid Accuracy: {correct.double()/len(valid_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(valid_loader.dataset)

In [22]:
    from torch.optim import AdamW
    from transformers import get_linear_schedule_with_warmup
    
    model = EquivalenceClassifier(n_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    # Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
    # Batchsize: 16, 32
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    lr_scheduler = get_linear_schedule_with_warmup(
                optimizer, 
                num_warmup_steps=0, 
                num_training_steps=len(train_loader)*EPOCHS
            )
    best_acc = 0
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print('-'*30)

        train(model, criterion, optimizer, train_loader)
        val_acc = eval()

        if val_acc > best_acc:
            torch.save(model.state_dict(), f'phobert_fold{epoch+1}.pth')
            best_acc = val_acc

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/6
------------------------------


100%|██████████| 7004/7004 [11:31<00:00, 10.13it/s]

Train Accuracy: 0.9426195129351502 Loss: 0.17243642459247477


Valid Accuracy: 0.9713299068422744 Loss: 0.10472053516199167
Epoch 2/6
------------------------------


...

Train Accuracy: 0.9944225809618148 Loss: 0.023462461626607388


Valid Accuracy: 0.986508191455188 Loss: 0.06953708278851477
Epoch 6/6
------------------------------


100%|██████████| 7004/7004 [11:28<00:00, 10.18it/s]

Train Accuracy: 0.9965643098724779 Loss: 0.014944094010944686


Valid Accuracy: 0.9870703501445551 Loss: 0.0785246220283939


In [56]:
from sklearn.metrics import classification_report, confusion_matrix

def test(data_loader):
    model = EquivalenceClassifier(n_classes=2)
    model.to(device)
    model.load_state_dict(torch.load(f'phobert_fold6.pth'))
    model.eval()

    texts1 = []
    texts2 = []
    predicts = []
    predict_probs = []
    real_values = []

    for data in tqdm(data_loader):
        text1 = data['text1']
        text2 = data['text2']
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        total_outs = []

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        total_outs.append(outputs)
        
        total_outs = torch.stack(total_outs)
        _, pred = torch.max(total_outs.mean(0), dim=1)
        texts1.extend(text1)
        texts2.extend(text2)
        predicts.extend(pred)
        predict_probs.extend(total_outs.mean(0))
        real_values.extend(targets)
    
    predicts = torch.stack(predicts).cpu()
    predict_probs = torch.stack(predict_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    correct_cases = 0
    for i in range(len(real_values)):
      if real_values[i] == predicts[i]:
        correct_cases += 1
    print(f'Test Accuracy: {correct_cases/len(real_values)}')
    print(classification_report(real_values, predicts))
    return real_values, predicts

In [64]:
test_dataset = EquavilenceDataset(test_df, tokenizer, max_len=50)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
real_values, predicts = test(test_loader)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 775/775 [00:12<00:00, 63.09it/s]


Test Accuracy: 0.9892655367231639
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     10759
           1       0.94      0.98      0.96      1631

    accuracy                           0.99     12390
   macro avg       0.97      0.99      0.98     12390
weighted avg       0.99      0.99      0.99     12390



In [66]:
for data in test_loader:
        text1 = data['text1']
        text2 = data['text2']
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)
        print(text1)
        print(targets)

['văn_hoá việt_nam có từ bao_giờ', 'việt_nam giáp với những quốc_gia và vùng lãnh_thổ nào', 'viết đầy_đủ của wto', 'album đầu_tay của adele tên gì', 'việt_nam có những thành_phố trực_thuộc trung_ương nào', 'độ lệch tâm quỹ_đạo của sao_hoả là bao_nhiêu', 'mô_hình and đầu_tiên do ai tìm ra', 'đền hoa sen được hoàn_thành vào năm nào', 'thời vua nào cho xây_dựng cầu_trường tiền', 'nhà_nước việt_nam đầu_tiên bắt_đầu khi nào', 'ronaldo giành hat trick tổng_cộng bao_nhiêu lần', 'việt_nam đồng được phát_hành lần đầu năm nào', 'vị vua tiếp_theo nhà hậu lê_sau vua lê_huyền tông là ai', 'câu_lạc_bộ bóng_đá nào nhiều lần vô_địch uefa champion league nhất', 'ngày truyền_thống cảnh_sát nhân_dân là ngày nào', 'vũ nhôm là biệt_danh của ai']
tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
['vua trần nhân tông đi_tu ở đâu', 'tử cấm_thành được unesco công_nhận di_sản thế_giới vào năm nào', 'công_thức_hoá_học của đá_vôi là gì', 'độ_ẩm_không_khí ở thành_phố thái_bình dao_động bao_

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


['việt_nam có bao_nhiêu km đường biên_giới với lào', 'tại_sao tỉnh hồ nam có tên là hồ nam', 'ba_lan rộng bao_nhiêu', 'ai được xem là vị thám_hoa đầu_tiên trong lịch_sử khoa_bảng nước ta', 'việt minh được thành_lập vào ngày_tháng năm nào', 'nước ta có bao_nhiêu tỉnh thành_phố ở miền trung', 'tiểu_thuyết anh_hùng xạ điêu của tác_giả nào', 'lê_đức anh mất năm bao_nhiêu', 'viết tắt của hệ_thống định_vị_toàn_cầu', 'tỉnh palawan còn có đảo nào khác ngoài đảo palawan là gì', 'nhật_bản có số dân là bao_nhiêu 2017', 'chùa từ hiếu thuộc tỉnh nào', 'nguyễn_trãi quê ở huyện nào', 'world_cup năm 2010 được tổ_chức ở quốc_gia nào', 'tên gọi tiếng hàn_quốc của cá_ngừ sọc dưa là gì', 'cung_điện mùa đông nằm ở thành_phố nào của liên xô']
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
['hiện_nay ngôn_ngữ nào là quốc_ngữ ở nam_phi', 'toà tháp nào nghiêng nhiều nhất thế_giới', 'độ cao của núi phú sĩ là bao_nhiêu', 'số người tham_dự mít_tinh độc_lập ngày 2 tháng 9 năm 1945 là bao

In [67]:
print(real_values[:32])

tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0])


In [76]:
print(real_values.unique(return_counts=True))

(tensor([0, 1]), tensor([10759,  1631]))


In [77]:
print(predicts.unique(return_counts=True))

(tensor([0, 1]), tensor([10690,  1700]))


In [112]:
count_true_wrong = 0
for i in range(len(test_df)):
    if test_df['similarity'][i] == True and predicts[i] == 0:
      print(test_df['question 1'][i]) 
      print(test_df['question 2'][i])
      print(test_df['similarity'][i]) 
      print(real_values[i])
      print(predicts[i])
      count_true_wrong += 1

print(f'Incorrect True label: {count_true_wrong}')

phố hiến xưa thuộc tỉnh nào
phố hiến ở tỉnh nào
True
tensor(1)
tensor(0)
vua lý_thái tổ đóng_đô ở đâu
lý thái tổ đóng_đô ở đâu
True
tensor(1)
tensor(0)
trạng bùng làm quan ở thời nào
phùng khắc khoan làm quan cho triều nào
True
tensor(1)
tensor(0)
đào an thái đã từng giữ những chức_vụ gì
đào an thái hiện đang giữ chức_vụ gì
True
tensor(1)
tensor(0)
ai là người biên_soạn đầu_tiên của đại việt sử_ký toàn thư
ai là người biên_soạn đại việt sử_ký toàn thư
True
tensor(1)
tensor(0)
ai là người phát_minh ra blockchain
blockchain do ai tạo ra
True
tensor(1)
tensor(0)
nhật_bản giáp với những nước nào
nhật_bản có vị_trí địa_lý tiếp_giáp với những nước nào
True
tensor(1)
tensor(0)
ronaldo sinh ra ở đâu
cristiano ronaldo sinh ra tại đâu
True
tensor(1)
tensor(0)
ngô_quyền đánh với quân nào trên sông bạch đằng
ngô_quyền đánh tan quân nào
True
tensor(1)
tensor(0)
nhà mặc thành_lập năm nào
mạc đăng dung cướp ngôi nhà hậu lê_lập ra nhà mạc năm bao_nhiêu
True
tensor(1)
tensor(0)
việt_nam dân_chủ cộng_ho

In [114]:
count_false_incorrect = 0
for i in range(len(test_df)):
    if test_df['similarity'][i] == False and predicts[i] == 1:
      print(test_df['question 1'][i]) 
      print(test_df['question 2'][i])
      print(test_df['similarity'][i]) 
      print(real_values[i])
      print(predicts[i])
      count_false_incorrect += 1

print(f'Incorrect False label: {count_false_incorrect}')

sông nội_địa nào dài nhất việt_nam
sông gì dài nhất việt_nam
False
tensor(0)
tensor(1)
hội việt_nam cách_mạng thanh_niên ra_đời vào năm nào
hội việt_nam cách_mạng thanh_niên ra_đời ngày nào
False
tensor(0)
tensor(1)
lãnh_thổ mỹ từng là thuộc địa của nước nào
mỹ từng là thuộc địa của nước nào
False
tensor(0)
tensor(1)
trái_đất xoay quanh mặt_trời với chu_kì là
năm thường được tính là khoảng thời_gian trái_đất quay xong mấy vòng_quanh mặt_trời
False
tensor(0)
tensor(1)
cầu kiền nằm trên đường nào
cầu kiền ở đâu
False
tensor(0)
tensor(1)
đại_tướng đầu_tiên của việt_nam tên là gì
đại_tướng đầu_tiên của nước ta là ai
False
tensor(0)
tensor(1)
bí_thư của thành_phố huế là người nào
ai là bí_thư của thành_phố huế
False
tensor(0)
tensor(1)
thái_bình_dương ngăn_cách các châu_lục nào
thái_bình_dương tiếp_giáp với các châu_lục nào
False
tensor(0)
tensor(1)
hai_bà_trưng sinh ra ở đâu
phường hai_bà_trưng thuộc tỉnh nào
False
tensor(0)
tensor(1)
tháp rùa có từ khi nào
tháp rùa có từ thời nào
False
te

In [91]:
class_names = ['False', 'True']

def infer(text1, text2, tokenizer, max_len=128):
    text1 = word_segment(text1)
    text2 = word_segment(text2)
    encoded_review = tokenizer.encode_plus(
        text1,
        text2,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    print(y_pred)
    print(f'Text1: {text1}')
    print(f'Text2: {text2}')
    print(f'Sentiment: {class_names[y_pred]}')

In [115]:
text1 = "nước chiếm bao nhiêu phần trăm trong cơ thể con người"
text2 = "nước chiếm bao nhiều phần trăm cơ thể"

In [116]:
infer(text1, text2, tokenizer)

tensor([1], device='cuda:0')
Text1: nước chiếm bao_nhiêu phần_trăm trong cơ_thể con_người
Text2: nước chiếm bao nhiều phần_trăm cơ_thể
Sentiment: True


In [117]:
!git init

Initialized empty Git repository in /content/.git/
